In [1]:
# Install CLIP library
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-thb4ty_p
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-thb4ty_p
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import json
import pandas as pd
import requests
import os
import io
from PIL import Image
from tqdm import tqdm
import pickle
from collections import defaultdict
import gc
import psutil
import shutil
import zipfile

# Paths
metadata_path = r"/kaggle/input/amazon-dataset/meta_Clothing_Shoes_and_Jewelry.json"
reviews_path = r"/kaggle/input/amazon-dataset/Clothing_Shoes_and_Jewelry.json"
output_pickle = r"/kaggle/working/preprocessed_fashion_data.pkl"
output_interactions = r"/kaggle/working/user_item_interactions.pkl"
image_folder = "/kaggle/working/images"
output_zip = "/kaggle/working/amazon_fashion_data.zip"

# Clean image folder to free disk space
if os.path.exists(image_folder):
    shutil.rmtree(image_folder)
os.makedirs(image_folder, exist_ok=True)

# Helper: Print memory usage
def print_memory_usage(step=""):
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory usage after {step}: {mem_info.rss / 1024 ** 2:.2f} MB")

# Helper: Download and save one image per item
def download_first_image(urls, asin):
    for i, url in enumerate(urls):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                img = Image.open(io.BytesIO(response.content))
                img.verify()  # Verify image integrity
                img_path = os.path.join(image_folder, f"{asin}.jpg")
                with open(img_path, 'wb') as f:
                    f.write(response.content)
                return img_path, url
        except Exception:
            continue
    return None, None

# Step 1: Load and filter metadata
print("Loading metadata...")
meta_asin_to_data = {}
valid_asins = set()
with open(metadata_path, 'r', encoding='utf-8') as f:
    for line in tqdm(f, desc="Reading metadata"):
        data = json.loads(line)
        asin = data.get('asin')
        if not asin:
            continue
        title = data.get('title', '')
        description = data.get('description', '')
        if isinstance(description, list):
            description = ' '.join([str(item) for item in description if item])
        elif not isinstance(description, str):
            description = ''
        brand = data.get('brand', '')
        categories = data.get('categories', [])
        if isinstance(categories, list):
            categories = ' '.join([str(cat) for sublist in categories for cat in sublist if cat])
        else:
            categories = ''
        image_urls = data.get('imageURLHighRes', [])
        textual_features = ' '.join([title, description, brand, categories]).strip()
        if textual_features and image_urls:
            meta_asin_to_data[asin] = {
                'textual_features': textual_features,
                'imageURLs': image_urls,
            }
            valid_asins.add(asin)
print(f"Found {len(valid_asins)} items with textual features and image URLs")
print_memory_usage("metadata loading")
del data, line
gc.collect()

# Step 2: Load reviews and include overall rating
print("Loading reviews...")
chunk_size = 50000
all_reviews = []
with open(reviews_path, 'r', encoding='utf-8') as f:
    reviews_chunk = []
    for line in tqdm(f, desc="Reading reviews"):
        data = json.loads(line)
        asin = data.get('asin')
        if asin in valid_asins:
            reviews_chunk.append({
                'reviewerID': data.get('reviewerID'),
                'asin': asin,
                'overall': data.get('overall', 0.0),
            })
        if len(reviews_chunk) >= chunk_size:
            df_chunk = pd.DataFrame(reviews_chunk)
            reviews_chunk = []  # Clear chunk
            # Step 3: Filter users and items with >=5 interactions
            user_counts = df_chunk['reviewerID'].value_counts()
            item_counts = df_chunk['asin'].value_counts()
            df_chunk = df_chunk[
                (df_chunk['reviewerID'].isin(user_counts[user_counts >= 5].index)) &
                (df_chunk['asin'].isin(item_counts[item_counts >= 5].index))
            ]
            if not df_chunk.empty:
                # Step 4: Merge metadata
                df_chunk['textual_features'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['textual_features'])
                df_chunk['imageURLs'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['imageURLs'])
                all_reviews.extend(df_chunk.to_dict('records'))
            del df_chunk, user_counts, item_counts
            print_memory_usage("processing review chunk")
            gc.collect()
    if reviews_chunk:
        df_chunk = pd.DataFrame(reviews_chunk)
        user_counts = df_chunk['reviewerID'].value_counts()
        item_counts = df_chunk['asin'].value_counts()
        df_chunk = df_chunk[
            (df_chunk['reviewerID'].isin(user_counts[user_counts >= 5].index)) &
            (df_chunk['asin'].isin(item_counts[item_counts >= 5].index))
        ]
        if not df_chunk.empty:
            df_chunk['textual_features'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['textual_features'])
            df_chunk['imageURLs'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['imageURLs'])
            all_reviews.extend(df_chunk.to_dict('records'))
        del df_chunk, user_counts, item_counts, reviews_chunk
        print_memory_usage("processing final review chunk")
        gc.collect()

# Build DataFrame
print("Building DataFrame...")
df = pd.DataFrame(all_reviews)
print_memory_usage("building DataFrame")
del all_reviews
gc.collect()

# Step 5: Download one image per item
print("Downloading one image per item...")
asin_to_image = {}
unique_asins = df['asin'].unique()
for asin in tqdm(unique_asins, desc="Downloading images"):
    urls = meta_asin_to_data[asin]['imageURLs']
    image_path, image_url = download_first_image(urls, asin)
    if image_path and image_url:
        asin_to_image[asin] = {'image_path': image_path, 'image_url': image_url}

# Filter DataFrame to keep only items with downloadable images
print("Filtering items without downloadable images...")
df = df[df['asin'].isin(asin_to_image.keys())].reset_index(drop=True)
df['image_path'] = df['asin'].map(lambda x: asin_to_image[x]['image_path'])
df['image_url'] = df['asin'].map(lambda x: asin_to_image[x]['image_url'])
print(f"Dataset size after image filtering: {len(df)} samples")
print_memory_usage("image downloading and filtering")
del asin_to_image
gc.collect()

# Verify image storage
print("Verifying image storage...")
image_count = len([f for f in os.listdir(image_folder) if f.endswith('.jpg')])
print(f"Total images stored: {image_count}")
if image_count == 0:
    print("Warning: No images were stored. Check image URLs or network connectivity.")

# Step 6: Build user-item interaction dictionary
print("Building user-item interaction dictionary...")
user_item_dict = defaultdict(list)
for idx, row in df.iterrows():
    user_item_dict[row['reviewerID']].append(
        (row['asin'], row['overall'], row['textual_features'], row['image_path'])
    )
print_memory_usage("building interactions")
gc.collect()

# Save processed DataFrame
print(f"Saving processed DataFrame to {output_pickle}...")
df.to_pickle(output_pickle)
print_memory_usage("saving DataFrame")
gc.collect()

# Save user-item interactions
print(f"Saving user-item interactions to {output_interactions}...")
with open(output_interactions, 'wb') as f:
    pickle.dump(user_item_dict, f)
print_memory_usage("saving interactions")
gc.collect()

# Create ZIP file of all outputs
print(f"Creating ZIP file: {output_zip}...")
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(image_folder):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, "/kaggle/working")
            zipf.write(file_path, arcname)
            print(f"Added {arcname} to ZIP")
    
    for file in ["preprocessed_fashion_data.pkl", "user_item_interactions.pkl"]:
        file_path = os.path.join("/kaggle/working", file)
        arcname = os.path.relpath(file_path, "/kaggle/working")
        zipf.write(file_path, arcname)
        print(f"Added {arcname} to ZIP")

# Verify ZIP file size
zip_size = os.path.getsize(output_zip) / (1024 ** 3)  # Size in GiB
print(f"ZIP file created: {output_zip} (Size: {zip_size:.2f} GiB)")
print("You can now download the ZIP file from the Output tab in Kaggle.")

print(f"✅ Done! Final dataset size: {len(df)} samples, {len(user_item_dict)} users")

Loading metadata...


Reading metadata: 2685059it [02:24, 18547.69it/s]


Found 1998477 items with textual features and image URLs
Memory usage after metadata loading: 4200.05 MB
Loading reviews...


Reading reviews: 71806it [00:02, 22307.27it/s] 

Memory usage after processing review chunk: 4203.12 MB


...

Reading reviews: 32264588it [22:59, 18813.57it/s]

Memory usage after processing review chunk: 4210.04 MB


Reading reviews: 32292099it [22:59, 23408.27it/s]


Memory usage after processing final review chunk: 4210.04 MB
Building DataFrame...
Memory usage after building DataFrame: 4210.04 MB


Filtering items without downloadable images...
Dataset size after image filtering: 14486 samples
Memory usage after image downloading and filtering: 4210.41 MB
Verifying image storage...
Total images stored: 6306
Building user-item interaction dictionary...
Memory usage after building interactions: 4210.41 MB
Saving processed DataFrame to /kaggle/working/preprocessed_fashion_data.pkl...
Memory usage after saving DataFrame: 4210.41 MB
Saving user-item interactions to /kaggle/working/user_item_interactions.pkl...
Memory usage after saving interactions: 4210.41 MB
Creating ZIP file: /kaggle/working/amazon_fashion_data.zip...
Added images/B00SQ3J25W.jpg to ZIP
Added images/B002WQ5YNS.jpg to ZIP
Added images/B000UPTVKA.jpg to ZIP
Added images/B006JWRDJ4.jpg to ZIP
Added images/B00XHLNA5G.jpg to ZIP
Added images/B0085M5K64.jpg to ZIP
Added images/B0017NQWF6.jpg to ZIP
Added images/B00JGHLG38.jpg to ZIP
Added images/B0012DHHJ6.jpg to ZIP
Added images/B01DVZZS8Y.jpg to ZIP
Added images/B01568Z

In [ ]:
import json
import pandas as pd
import requests
import os
import io
from PIL import Image
from tqdm import tqdm
import pickle
from collections import defaultdict
import gc
import psutil
import random
import shutil
import zipfile

# Paths
metadata_path = r"/kaggle/input/amazon-dataset/meta_Clothing_Shoes_and_Jewelry.json"
reviews_path = r"/kaggle/input/amazon-dataset/Clothing_Shoes_and_Jewelry.json"
output_pickle = r"/kaggle/working/preprocessed_fashion_data.pkl"
output_interactions = r"/kaggle/working/user_item_interactions.pkl"
image_folder = "/kaggle/working/images"
output_zip = "/kaggle/working/amazon_fashion_data.zip"

# Clean image folder to free disk space
if os.path.exists(image_folder):
    shutil.rmtree(image_folder)
os.makedirs(image_folder, exist_ok=True)

# Helper: Print memory usage
def print_memory_usage(step=""):
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory usage after {step}: {mem_info.rss / 1024 ** 2:.2f} MB")

# Helper: Download and save image to disk
def download_and_save_images(urls, asin, idx):
    paths = []
    for i, url in enumerate(urls):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                img = Image.open(io.BytesIO(response.content))
                img.verify()  # Verify image integrity
                img_path = os.path.join(image_folder, f"{asin}_{idx}_{i}.jpg")
                with open(img_path, 'wb') as f:
                    f.write(response.content)
                paths.append(img_path)
        except Exception:
            continue
    return paths

# Load metadata with image and description filtering
print("Loading metadata...")
meta_asin_to_data = {}
valid_asins = set()
with open(metadata_path, 'r', encoding='utf-8') as f:
    for line in tqdm(f, desc="Reading metadata"):
        data = json.loads(line)
        asin = data.get('asin')
        if not asin:
            continue
        description = data.get('description', '')
        if isinstance(description, list):
            description = ' '.join([str(item) for item in description if item])
        elif not isinstance(description, str):
            description = ''
        image_urls = data.get('imageURLHighRes', [])
        # Keep items with non-empty description and at least one image URL
        if description.strip() and image_urls:
            meta_asin_to_data[asin] = {
                'title': data.get('title', ''),
                'description': description,
                'feature': ' '.join(data.get('feature', [])),
                'imageURLs': image_urls,
            }
            valid_asins.add(asin)
print(f"Found {len(valid_asins)} items with description and image URLs")
print_memory_usage("metadata loading")
del data, line
gc.collect()

# Load reviews in chunks, keeping only valid ASINs
print("Loading reviews...")
chunk_size = 50000
all_reviews = []  # Renamed to avoid confusion
with open(reviews_path, 'r', encoding='utf-8') as f:
    reviews_chunk = []
    for line in tqdm(f, desc="Reading reviews"):
        data = json.loads(line)
        asin = data.get('asin')
        if asin in valid_asins:
            reviews_chunk.append({
                'reviewerID': data.get('reviewerID'),
                'asin': asin,
                'reviewText': data.get('reviewText', ''),
                'summary': data.get('summary', ''),
            })
        if len(reviews_chunk) >= chunk_size:
            df_chunk = pd.DataFrame(reviews_chunk)
            reviews_chunk = []  # Clear chunk
            # Filter users and items with >=5 interactions
            user_counts = df_chunk['reviewerID'].value_counts()
            item_counts = df_chunk['asin'].value_counts()
            df_chunk = df_chunk[
                (df_chunk['reviewerID'].isin(user_counts[user_counts >= 5].index)) &
                (df_chunk['asin'].isin(item_counts[item_counts >= 5].index))
            ]
            if not df_chunk.empty:
                # Merge metadata
                df_chunk['title'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['title'])
                df_chunk['description'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['description'])
                df_chunk['feature'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['feature'])
                df_chunk['imageURLs'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['imageURLs'])
                # Concatenate text
                df_chunk['concatenated_text'] = [
                    ' '.join([
                        row['title'] or '',
                        row['description'] or '',
                        row['feature'] or '',
                        row['summary'] or '',
                        row['reviewText'] or ''
                    ]) for _, row in df_chunk.iterrows()
                ]
                all_reviews.extend(df_chunk.to_dict('records'))
            del df_chunk, user_counts, item_counts
            print_memory_usage("processing review chunk")
            gc.collect()
    # Process remaining reviews
    if reviews_chunk:
        df_chunk = pd.DataFrame(reviews_chunk)
        user_counts = df_chunk['reviewerID'].value_counts()
        item_counts = df_chunk['asin'].value_counts()
        df_chunk = df_chunk[
            (df_chunk['reviewerID'].isin(user_counts[user_counts >= 5].index)) &
            (df_chunk['asin'].isin(item_counts[item_counts >= 5].index))
        ]
        if not df_chunk.empty:
            df_chunk['title'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['title'])
            df_chunk['description'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['description'])
            df_chunk['feature'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['feature'])
            df_chunk['imageURLs'] = df_chunk['asin'].map(lambda x: meta_asin_to_data[x]['imageURLs'])
            df_chunk['concatenated_text'] = [
                ' '.join([
                    row['title'] or '',
                    row['description'] or '',
                    row['feature'] or '',
                    row['summary'] or '',
                    row['reviewText'] or ''
                ]) for _, row in df_chunk.iterrows()
            ]
            all_reviews.extend(df_chunk.to_dict('records'))
        del df_chunk, user_counts, item_counts, reviews_chunk
        print_memory_usage("processing final review chunk")
        gc.collect()

# Build dataframe
print("Building dataframe...")
df = pd.DataFrame(all_reviews)
print_memory_usage("building dataframe")
del all_reviews
gc.collect()

# Filter to 5000 users and 4000 items
print("Selecting 5000 users and 4000 items...")
user_counts = df['reviewerID'].value_counts()
item_counts = df['asin'].value_counts()
valid_users = user_counts[user_counts >= 5].index
valid_items = item_counts[item_counts >= 5].index

# Randomly sample 5000 users and 4000 items
random.seed(42)  # For reproducibility
selected_users = random.sample(list(valid_users), min(5000, len(valid_users)))
selected_items = random.sample(list(valid_items), min(4000, len(valid_items)))

df = df[
    (df['reviewerID'].isin(selected_users)) &
    (df['asin'].isin(selected_items))
].reset_index(drop=True)
print_memory_usage("user/item filtering")
del user_counts, item_counts, valid_users, valid_items, selected_users, selected_items
gc.collect()

# Download and save images
print("Downloading and saving images...")
image_paths = []
for idx, (urls, asin) in tqdm(enumerate(zip(df['imageURLs'], df['asin'])), total=len(df)):
    paths = download_and_save_images(urls, asin, idx)
    image_paths.append(paths)
df['image_paths'] = image_paths
print_memory_usage("image downloading")
gc.collect()

# Verify image storage
print("Verifying image storage...")
image_count = len([f for f in os.listdir(image_folder) if f.endswith('.jpg')])
print(f"Total images stored: {image_count}")
if image_count == 0:
    print("Warning: No images were stored. Check image URLs or network connectivity.")

# Filter samples with at least one image
print("Filtering samples without images...")
df = df[df['image_paths'].apply(lambda x: len(x) > 0)].reset_index(drop=True)
print(f"Dataset size after image filtering: {len(df)} samples")
print_memory_usage("filtering images")
gc.collect()

# Build user-item interaction dictionary
print("Building user-item interaction dictionary...")
user_item_dict = defaultdict(list)
for idx, row in df.iterrows():
    user_item_dict[row['reviewerID']].append(
        (row['asin'], row['concatenated_text'], row['image_paths'])
    )
print_memory_usage("building interactions")
gc.collect()

# Save processed dataframe
print(f"Saving processed dataframe to {output_pickle}...")
df.to_pickle(output_pickle)
print_memory_usage("saving dataframe")
gc.collect()

# Save user-item interactions
print(f"Saving user-item interactions to {output_interactions}...")
with open(output_interactions, 'wb') as f:
    pickle.dump(user_item_dict, f)
print_memory_usage("saving interactions")
gc.collect()

# Create ZIP file of all outputs in /kaggle/working
print(f"Creating ZIP file: {output_zip}...")
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add the images folder
    for root, _, files in os.walk(image_folder):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, "/kaggle/working")
            zipf.write(file_path, arcname)
            print(f"Added {arcname} to ZIP")
    
    # Add the pickle files
    for file in ["preprocessed_fashion_data.pkl", "user_item_interactions.pkl"]:
        file_path = os.path.join("/kaggle/working", file)
        arcname = os.path.relpath(file_path, "/kaggle/working")
        zipf.write(file_path, arcname)
        print(f"Added {arcname} to ZIP")

# Verify ZIP file size
zip_size = os.path.getsize(output_zip) / (1024 ** 3)  # Size in GiB
print(f"ZIP file created: {output_zip} (Size: {zip_size:.2f} GiB)")
print("You can now download the ZIP file from the Output tab in Kaggle.")

# Optional: Free disk space by deleting images folder
# shutil.rmtree(image_folder)
# print("Deleted images folder to free disk space.")

print(f"✅ Done! Final dataset size: {len(df)} samples, {len(user_item_dict)} users")